In [1]:
from pandas import DataFrame,read_csv,read_excel,merge,melt,pivot_table
import numpy
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb
import sys
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from factor_analyzer.factor_analyzer import calculate_kmo

## 데이터 준비

In [2]:
origin=read_excel('행정동병합테스트22.xlsx')
origin.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,아동인구,노인인구,장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,612,2154,437,337.55,4,0.352898,0,32,0,3
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,488,1808,267,60.79,0,0.063554,0,27,0,12
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,108,626,110,956.51,3,1.000000,0,16,0,5
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,385,1814,311,0.00,3,0.000000,10,34,0,5
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,1033,3495,491,1.14,3,0.001192,10,54,0,5


In [3]:
origin.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계_명', '면적', '구성비', '행정동별_공원수',
       '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소', '면수', '하천및한강여부', '아동인구',
       '노인인구', '장애인_인구', '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수',
       '행정동별버스정류장수', '행정동별유통업체수', '공공자전거대여소수'],
      dtype='object')

## 결측치는 0으로 처리

> 실제 조사된 값이므로 없다면 관측되지 아니함으로 판단

In [4]:
origin.isna().sum()
origin1=origin.fillna(0)

In [5]:
origin1.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계_명', '면적', '구성비', '행정동별_공원수',
       '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소', '면수', '하천및한강여부', '아동인구',
       '노인인구', '장애인_인구', '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수',
       '행정동별버스정류장수', '행정동별유통업체수', '공공자전거대여소수'],
      dtype='object')

## 데이터 스케일링

In [6]:
scaler=MinMaxScaler()
df = origin1.copy()
df['유통업체minmax']=scaler.fit_transform(df[['행정동별유통업체수']])
df['공공체육시설minmax']=scaler.fit_transform(df[['공공체육시설_수']])
df['문화시설minmax']=scaler.fit_transform(df[['문화시설_수']])
df['면적minmax']=scaler.fit_transform(df[['면적']])
df.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,유통업체minmax,공공체육시설minmax,문화시설minmax,면적minmax
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,4,0.352898,0,32,0,3,0.0,0.256410,0.055556,0.187952
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,0,0.063554,0,27,0,12,0.0,0.205128,0.120370,0.080321
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,3,1.000000,0,16,0,5,0.0,0.051282,0.287037,0.101205
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,3,0.000000,10,34,0,5,0.0,0.128205,0.046296,0.163855
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,3,0.001192,10,54,0,5,0.0,0.128205,0.101852,0.693976


## 파생변수(`여가생활지수`) 생성

In [7]:
df['여가생활지수'] = (df['공공체육시설minmax'] + df['문화시설minmax'] + df['행정동별유통업체수']) / df['면적minmax']
df.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,유통업체minmax,공공체육시설minmax,문화시설minmax,면적minmax,여가생활지수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,0.352898,0,32,0,3,0.0,0.256410,0.055556,0.187952,1.659818
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,0.063554,0,27,0,12,0.0,0.205128,0.120370,0.080321,4.052457
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,1.000000,0,16,0,5,0.0,0.051282,0.287037,0.101205,3.342915
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,0.000000,10,34,0,5,0.0,0.128205,0.046296,0.163855,1.064972
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,0.001192,10,54,0,5,0.0,0.128205,0.101852,0.693976,0.331506


## 파생 변수의 값 정렬

In [8]:
df['여가생활지수'].value_counts().sort_index()

여가생활지수
0.000000      37
0.050560       1
0.060006       1
0.067414       1
0.103854       1
              ..
292.941176     1
359.007937     1
373.500000     1
626.933761     1
inf            1
Name: count, Length: 380, dtype: int64

## 결측치 확인

In [9]:
df['여가생활지수'].isna().sum()

1

In [10]:
# df['여가생활지수'].filln(0)
# df['여가생활지수'].isnu().sum()

In [11]:
df['여가생활지수'].unique()

array([1.65981810e+00, 4.05245726e+00, 3.34291480e+00, 1.06497193e+00,
       3.31505718e-01, 9.82248521e+00, 3.62090456e+00, 2.23118280e+00,
       4.32835430e+01, 1.01880631e+02, 2.49580420e+01, 2.22784020e+01,
       1.65046741e+02, 0.00000000e+00,            inf,            nan,
       1.03750000e+02, 5.36361883e+01, 1.84589814e+02, 9.93512427e+01,
       3.80036630e-01, 2.38559867e+00, 2.92941176e+02, 1.01257508e+02,
       6.26933761e+02, 1.23511905e+00, 1.01338034e+02, 3.95007770e+00,
       1.44097222e+00, 3.84259259e+00, 3.73500000e+02, 1.27692308e+00,
       6.89696106e-01, 1.14538818e+00, 1.16468937e+00, 1.77350427e+00,
       1.99519231e+00, 1.06410256e+01, 6.07475031e+01, 2.86592896e-01,
       3.69566993e+01, 1.82980600e-01, 7.81490192e-01, 4.88921314e+00,
       2.56434240e+01, 6.70471128e-01, 1.10079576e+00, 7.04475309e+01,
       1.31052632e+02, 9.38914027e-01, 5.01207729e-01, 1.05977694e+00,
       6.74139051e-02, 6.82901071e-01, 1.74663300e-01, 1.32446809e+01,
      

In [12]:
df.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계_명', '면적', '구성비', '행정동별_공원수',
       '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소', '면수', '하천및한강여부', '아동인구',
       '노인인구', '장애인_인구', '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수',
       '행정동별버스정류장수', '행정동별유통업체수', '공공자전거대여소수', '유통업체minmax', '공공체육시설minmax',
       '문화시설minmax', '면적minmax', '여가생활지수'],
      dtype='object')

## 지수 파일 생성에 사용될 컬럼 추출

In [13]:
df1=df.filter(['시군구코드','시군구명','행정동코드','행정동코드수정','읍면동명','공공체육시설_수','문화시설_수','면적', '행정동별유통업체수', '공공체육시설minmax','문화시설minmax','면적minmax', '유통업체minmax', '여가생활지수'])
df1.head()

,시군구코드,시군구명,행정동코드,읍면동명,공공체육시설_수,문화시설_수,면적,행정동별유통업체수,공공체육시설minmax,문화시설minmax,면적minmax,유통업체minmax,여가생활지수
0,11110,종로구,1111051500,청운효자동,10,6,2.57,0,0.256410,0.055556,0.187952,0.0,1.659818
1,11110,종로구,1111053000,사직동,8,13,1.23,0,0.205128,0.120370,0.080321,0.0,4.052457
2,11110,종로구,1111054000,삼청동,2,31,1.49,0,0.051282,0.287037,0.101205,0.0,3.342915
3,11110,종로구,1111055000,부암동,5,5,2.27,0,0.128205,0.046296,0.163855,0.0,1.064972
4,11110,종로구,1111056000,평창동,5,11,8.87,0,0.128205,0.101852,0.693976,0.0,0.331506


## 지수 파일 생성

In [14]:
df1.to_excel('여가생활지수1.xlsx', index=False)

In [15]:
df2=read_excel('여가생활지수1.xlsx')
df2.head()

,시군구코드,시군구명,행정동코드,읍면동명,공공체육시설_수,문화시설_수,면적,행정동별유통업체수,공공체육시설minmax,문화시설minmax,면적minmax,유통업체minmax,여가생활지수
0,11110,종로구,1111051500,청운효자동,10,6,2.57,0,0.256410,0.055556,0.187952,0.0,1.659818
1,11110,종로구,1111053000,사직동,8,13,1.23,0,0.205128,0.120370,0.080321,0.0,4.052457
2,11110,종로구,1111054000,삼청동,2,31,1.49,0,0.051282,0.287037,0.101205,0.0,3.342915
3,11110,종로구,1111055000,부암동,5,5,2.27,0,0.128205,0.046296,0.163855,0.0,1.064972
4,11110,종로구,1111056000,평창동,5,11,8.87,0,0.128205,0.101852,0.693976,0.0,0.331506


## 이상치 `inf`, `-inf` 처리

In [16]:
# DataFrame(df2)에서 '여가생활지수' 열을 선택
accessibility_series = df2['여가생활지수']

# 'inf'와 '-inf'를 0으로 대체
accessibility_series = accessibility_series.replace([np.inf, -np.inf], 0)

# 변경된 '여가생활지수' 열을 DataFrame에 할당
df2['여가생활지수'] = accessibility_series

In [17]:
df2.to_excel("여가생활지수1.xlsx", index=False)

In [18]:
df2 = read_excel("여가생활지수1.xlsx")
df2.head()

,시군구코드,시군구명,행정동코드,읍면동명,공공체육시설_수,문화시설_수,면적,행정동별유통업체수,공공체육시설minmax,문화시설minmax,면적minmax,유통업체minmax,여가생활지수
0,11110,종로구,1111051500,청운효자동,10,6,2.57,0,0.256410,0.055556,0.187952,0.0,1.659818
1,11110,종로구,1111053000,사직동,8,13,1.23,0,0.205128,0.120370,0.080321,0.0,4.052457
2,11110,종로구,1111054000,삼청동,2,31,1.49,0,0.051282,0.287037,0.101205,0.0,3.342915
3,11110,종로구,1111055000,부암동,5,5,2.27,0,0.128205,0.046296,0.163855,0.0,1.064972
4,11110,종로구,1111056000,평창동,5,11,8.87,0,0.128205,0.101852,0.693976,0.0,0.331506


## 지수 정규화

In [20]:
df2['여가생활지수minmax'] = scaler.fit_transform(df2[['여가생활지수']])
df2

,시군구코드,시군구명,행정동코드,읍면동명,공공체육시설_수,문화시설_수,면적,행정동별유통업체수,공공체육시설minmax,문화시설minmax,면적minmax,유통업체minmax,여가생활지수,여가생활지수minmax
0,11110,종로구,1111051500,청운효자동,10,6,2.57,0,0.256410,0.055556,0.187952,0.0000,1.659818,0.002648
1,11110,종로구,1111053000,사직동,8,13,1.23,0,0.205128,0.120370,0.080321,0.0000,4.052457,0.006464
2,11110,종로구,1111054000,삼청동,2,31,1.49,0,0.051282,0.287037,0.101205,0.0000,3.342915,0.005332
3,11110,종로구,1111055000,부암동,5,5,2.27,0,0.128205,0.046296,0.163855,0.0000,1.064972,0.001699
4,11110,종로구,1111056000,평창동,5,11,8.87,0,0.128205,0.101852,0.693976,0.0000,0.331506,0.000529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,11740,강동구,1174065000,성내제2동,0,2,0.67,1,0.000000,0.018519,0.035341,0.0625,28.819444,0.045969
422,11740,강동구,1174066000,성내제3동,0,1,0.71,1,0.000000,0.009259,0.038554,0.0625,26.177662,0.041755
423,11740,강동구,1174068500,길동,1,1,1.61,2,0.025641,0.009259,0.110843,0.1250,18.358340,0.029283
424,11740,강동구,1174069000,둔촌제1동,0,0,0.92,0,0.000000,0.000000,0.055422,0.0000,0.000000,0.000000


## 최종 데이터 저장

In [ ]:
df2.to_excel('여가생활지수1.xlsx',index=False)